In [1]:
from thorlabs_apt_device.devices.bsc import BSC201 # https://thorlabs-apt-device.readthedocs.io/en/latest/

In [2]:
import serial
from serial.tools.list_ports import comports

In [3]:
import time

In [4]:
for i, p in enumerate(comports()):
    print(f'[{i}] {p}')

[0] /dev/ttyS0 - ttyS0
[1] /dev/ttyUSB2 - APT Stepper Motor Controller - APT Stepper Motor Controller
[2] /dev/ttyUSB1 - FT230X Basic UART - FT230X Basic UART


In [ ]:
# Sélection manuelle
appareil_apt = comports()[int(input('APT: '))]
appareil_elliptec = comports()[int(input('ELL: '))]

In [5]:
# Sélection automatique
motif_apt = 'APT Stepper Motor Controller'
motif_elliptec = 'FT230X Basic UART'

appareil_apt = next(filter(lambda x: motif_apt in str(x), comports()))
appareil_elliptec = next(filter(lambda x: motif_elliptec in str(x), comports()))

In [6]:
appareil_apt, appareil_elliptec

(<serial.tools.list_ports_linux.SysFS at 0x7f85ec474a90>,
 <serial.tools.list_ports_linux.SysFS at 0x7f85ec474be0>)

In [ ]:
numéro_série = "40219394" # Pour l'appareil spécifique qu'on utilise

# Essai avec la classe modifié

In [17]:
# Basé sur https://thorlabs-apt-device.readthedocs.io/en/latest/api/thorlabs_apt_device.devices.bsc.html#thorlabs_apt_device.devices.bsc.BSC201
class BSC201_HDR50(BSC201):
    # Données tirées de Kinesis
    kinesis = {'travel [pas]': 27_033_577,
               'vel [pas/s]': 201_578_964,
               'acc [pas/s²]': 20_653,
               'jog_step [pas]': 37_547}
    # Données tirées de https://www.thorlabs.com/drawings/8d6029bfe62e7acd-DF270E33-CE4C-A18B-8D52A70104946BCD/HDR50-Manual.pdf
    manuel = {'travel [°]': 360,
              'vel [°/s]': 50,
              'acc [°/s²]': 80,
              'macropas [°]': 1.8,
              'micropas [1/macropas]': 2048,
              'jog_step [pas]': kinesis['jog_step [pas]']}
    manuel['micropas [°]'] = manuel['macropas [°]'] / manuel['micropas [1/macropas]']
    manuel['travel [pas]'] = int( manuel['travel [°]'] / manuel['micropas [°]'] )
    assert manuel['travel [pas]'] == 409_600
    manuel['vel [pas/s]'] = int( manuel['vel [°/s]'] / manuel['micropas [°]'] )
    manuel['acc [pas/s²]'] = int( manuel['acc [°/s²]'] / manuel['micropas [°]'] )
    
    def __init__(self,
                 serial_port=None,
                 vid=None,
                 pid=None,
                 manufacturer=None,
                 product=None,
                 serial_number='40',
                 location=None,
                 home=True,
                 invert_direction_logic=False,
                 swap_limit_switches=True,
                 réglages=manuel):
        début = time.time()
        super().__init__(serial_port=serial_port,
                         vid=vid, pid=pid,
                         manufacturer=manufacturer,
                         product=product,
                         serial_number=serial_number,
                         location=location,
                         home=home,
                         invert_direction_logic=invert_direction_logic,
                         swap_limit_switches=swap_limit_switches)
    
        # Initial velocity parameters are effectively zero on startup, set something more sensible
        # Homing is initiated 1.0s after init, so hopefully these will take effect before then...
        # Tiré de la source pour BSC201_DRV250
        for bay_i, _ in enumerate(self.bays):
            for channel_i, _ in enumerate(self.channels):
                self.set_velocity_params(acceleration=réglages['acc [pas/s²]'],
                                         max_velocity=réglages['vel [pas/s]'],
                                         bay=bay_i,
                                         channel=channel_i)
                self.set_jog_params(size=réglages['jog_step [pas]'],
                                    acceleration=réglages['acc [pas/s²]'],
                                    max_velocity=réglages['vel [pas/s]'],
                                    bay=bay_i,
                                    channel=channel_i)
                self.set_home_params(velocity=réglages['vel [pas/s]'],
                                     offset_distance=0,
                                     bay=bay_i,
                                     channel=channel_i)
                # Use open-loop positioning (only using stepper counts)
                self.set_loop_params(loop_mode=1, prop=0, integral=0, diff=0, pid_clip=0, pid_tol=0, encoder_const=0, bay=0, channel=0)
        
        délai = time.time() - début
        if délai > 1:
            print(f"L'initialisation s'est faite trop lentement ({délai=}s).")


In [38]:
stage.close()

In [39]:
stage = BSC201_HDR50(serial_port=appareil_apt, réglages=BSC201_HDR50.manuel)
stage.identify()

In [47]:
stage.status

{'position': -27723,
 'enc_count': 0,
 'velocity': 0.0,
 'forward_limit_switch': True,
 'reverse_limit_switch': False,
 'moving_forward': False,
 'moving_reverse': True,
 'jogging_forward': False,
 'jogging_reverse': False,
 'motor_connected': False,
 'homing': False,
 'homed': False,
 'tracking': True,
 'interlock': False,
 'settled': False,
 'motion_error': False,
 'motor_current_limit_reached': False,
 'channel_enabled': True,
 'msg': 'mot_get_statusupdate',
 'msgid': 1153,
 'source': 33,
 'dest': 1,
 'chan_ident': 1,
 'forward_limit_soft': False,
 'reverse_limit_soft': False,
 'initializing': False,
 'instrument_error': False,
 'overtemp': False,
 'voltage_fault': False,
 'commutation_error': False,
 'digital_in_1': True,
 'digital_in_2': False,
 'digital_in_3': True,
 'digital_in_4': False,
 'encoder_fault': False,
 'overcurrent': False,
 'current_fault': False,
 'power_ok': False,
 'active': False,
 'error': False}

In [45]:
stage.move_velocity(True)
stage.status

{'position': -36743,
 'enc_count': 0,
 'velocity': 0.0,
 'forward_limit_switch': True,
 'reverse_limit_switch': False,
 'moving_forward': False,
 'moving_reverse': False,
 'jogging_forward': False,
 'jogging_reverse': False,
 'motor_connected': False,
 'homing': False,
 'homed': False,
 'tracking': True,
 'interlock': False,
 'settled': False,
 'motion_error': False,
 'motor_current_limit_reached': False,
 'channel_enabled': True,
 'msg': 'mot_get_statusupdate',
 'msgid': 1153,
 'source': 33,
 'dest': 1,
 'chan_ident': 1,
 'forward_limit_soft': False,
 'reverse_limit_soft': False,
 'initializing': False,
 'instrument_error': False,
 'overtemp': False,
 'voltage_fault': False,
 'commutation_error': False,
 'digital_in_1': True,
 'digital_in_2': False,
 'digital_in_3': True,
 'digital_in_4': False,
 'encoder_fault': False,
 'overcurrent': False,
 'current_fault': False,
 'power_ok': False,
 'active': False,
 'error': False}

In [49]:
stage.stop(True)
stage.status

{'position': -19751,
 'enc_count': 0,
 'velocity': 0.0,
 'forward_limit_switch': True,
 'reverse_limit_switch': False,
 'moving_forward': False,
 'moving_reverse': False,
 'jogging_forward': False,
 'jogging_reverse': False,
 'motor_connected': False,
 'homing': False,
 'homed': False,
 'tracking': True,
 'interlock': False,
 'settled': False,
 'motion_error': False,
 'motor_current_limit_reached': False,
 'channel_enabled': True,
 'msg': 'mot_get_statusupdate',
 'msgid': 1153,
 'source': 33,
 'dest': 1,
 'chan_ident': 1,
 'forward_limit_soft': False,
 'reverse_limit_soft': False,
 'initializing': False,
 'instrument_error': False,
 'overtemp': False,
 'voltage_fault': False,
 'commutation_error': False,
 'digital_in_1': True,
 'digital_in_2': False,
 'digital_in_3': True,
 'digital_in_4': False,
 'encoder_fault': False,
 'overcurrent': False,
 'current_fault': False,
 'power_ok': False,
 'active': False,
 'error': False}

In [50]:
stage.close()